# DeepGraphLibraly

## 1. グラフ  
グラフはentity（ノード）とrelation（エッジ）からなる。  
DGLでは、グラフの構造、ノードとエッジを定義し、実行できるライブラリである。

### 1.1. 基本的な定義について   
グラフは**entity**（ノード）と**relation**（エッジ）*を表すために用いられる構造である。  
それぞれの定義は次のようになる
> *__G=（V,E）__*  
> **V**＝ノードの集合  
> **E**＝エッジの集合   

たとえばエッジ *( u , v )*∈* E*はノード *u , v* がつながっていることを示す。  
また、グラフの**無向有向、同種異種、重みづけの有無**を定義することができる。  
**異種グラフ**はノード、エッジのタイプが異なるもの  
**マルチグラフ**は同じノード間に複数のエッジを許容するもの

### 1.2. グラフ、ノード、エッジ

ノードはそれぞれIDが割り当てられ、０から順に割り当てられる。  
複数のノードを指定するには*Pytorch、Tensorflow*のテンソルを用いる。  
**DGL**においてグラフを作成する際には `dgl.graph` を用いる  
![example](https://data.dgl.ai/asset/image/user_guide_graphch_1.png)

In [ ]:
import dgl
import torch as th

# edges 0->1, 0->2, 0->3, 1->3
u, v = th.tensor([0, 0, 0, 1]), th.tensor([1, 2, 3, 3])
g = dgl.graph((u, v))
print(g) # number of nodes are inferred from the max node IDs in the given edges
#Graph(num_nodes=4, num_edges=4,
#      ndata_schemes={}
#      edata_schemes={})

# Node IDs
print(g.nodes())
#tensor([0, 1, 2, 3])
# Edge end nodes
print(g.edges())
#(tensor([0, 0, 0, 1]), tensor([1, 2, 3, 3]))
# Edge end nodes and edge IDs
print(g.edges(form='all'))
#(tensor([0, 0, 0, 1]), tensor([1, 2, 3, 3]), tensor([0, 1, 2, 3]))

# If the node with the largest ID is isolated (meaning no edges),
# then one needs to explicitly set the number of nodes
g = dgl.graph((u, v), num_nodes=8)

仮に無向グラフを定義したい場合は `dgl.to_bidirected()` を用いる  
ノードID、エッジIDには32bit,64bit整数が使えるが、**型を統一する必要がある**  
デフォルトは64bitで変換するメゾットがある

### 1.3. ノード、エッジの機能

ノードとエッジにグラフ固有の特徴を格納するために、名前付きのプロパティを `ndata` 、 `edata` で定義できる。  
サンプルでは*x , y , z*のプロパティを定義した

In [ ]:
import dgl
import torch as th
g = dgl.graph(([0, 0, 1, 5], [1, 2, 2, 0])) # 6 nodes, 4 edges
g
#Graph(num_nodes=6, num_edges=4,
#      ndata_schemes={}
#      edata_schemes={})
g.ndata['x'] = th.ones(g.num_nodes(), 3)               # node feature of length 3
g.edata['x'] = th.ones(g.num_edges(), dtype=th.int32)  # scalar integer feature
g
#Graph(num_nodes=6, num_edges=4,
#      ndata_schemes={'x' : Scheme(shape=(3,), dtype=torch.float32)}
#      edata_schemes={'x' : Scheme(shape=(,), dtype=torch.int32)})
# different names can have different shapes
g.ndata['y'] = th.randn(g.num_nodes(), 5)
g.ndata['x'][1]                  # get node 1's feature
#tensor([1., 1., 1.])
g.edata['x'][th.tensor([0, 3])]  # get features of edge 0 and 3
#tensor([1, 1], dtype=torch.int32)

#### `edata`,`ndata` について
* 整数、ベクトル、テンソルのみが使える  
* ノード内のプロパティ名は重複できないが、ノード、エッジの間ではできる  
* プロパティはテンソルによって順に割り当てられていき、その次元数はノードorエッジの数と同じでなければいけない  
* 同一プロパティは同じ型でなければいけない  
* プロパティのテンソルは上のコードの最後の４行のように格納されている  

#### 重みの定義
グラフのエッジの重みは次のように定義できる

In [ ]:
# edges 0->1, 0->2, 0->3, 1->3
edges = th.tensor([0, 0, 0, 1]), th.tensor([1, 2, 3, 3])
weights = th.tensor([0.1, 0.6, 0.9, 0.7])  # weight of each edge
g = dgl.graph(edges)
g.edata['w'] = weights  # give it a name 'w'
g
#Graph(num_nodes=4, num_edges=4,
#      ndata_schemes={}
#      edata_schemes={'w' : Scheme(shape=(,), dtype=torch.float32)})

### 1.4. 外部ソースからグラフを作成
外部ソースからのグラフの構築する方法は  
* NetworkXやSciPyなどのライブラリからの変換  
* ローカルディスクから読み込む  

ほかにもいろいろ

#### NetworkX、Scipyから返還する方法

In [ ]:
import dgl
import torch as th
import scipy.sparse as sp
spmat = sp.rand(100, 100, density=0.05) # 5% nonzero entries
dgl.from_scipy(spmat)                   # from SciPy
#Graph(num_nodes=100, num_edges=500,
#      ndata_schemes={}
#      edata_schemes={})

import networkx as nx
nx_g = nx.path_graph(5) # a chain 0-1-2-3-4
dgl.from_networkx(nx_g) # from networkx
#Graph(num_nodes=5, num_edges=8,
#      ndata_schemes={}
#      edata_schemes={})

#### ディスクからのロード
特定のフォーマットで記述されたcsvファイルから `tensor` や `ndarray` に変換する。  
**pytorch**にはテンソルを保存 `torch,save` 、ロード `torch.load` するメゾットがある。

#### 他にも
`dgl.save_graphs()` `dgl.load_graphs()` によってグラフ自体を保存できる？

### 1.5. 異種グラフについて
ノード、エッジの種類が複数あり、関係が一般的な**トリプル**で示されるもの。無向グラフは、<span style="color:pink;">エッジが一種類かつ双方向の異種グラフである</span>ことに注意。  
ノード、エッジそれぞれIDで管理され、０から順に割り当てられる。  

#### グラフの作成
異種グラフは、接続関係ごとに次のように定義される

In [ ]:
import dgl
import torch as th

# Create a heterograph with 3 node types and 3 edges types.
graph_data = {
   ('drug', 'interacts', 'drug'): (th.tensor([0, 1]), th.tensor([1, 2])),
   ('drug', 'interacts', 'gene'): (th.tensor([0, 1]), th.tensor([2, 3])),
   ('drug', 'treats', 'disease'): (th.tensor([1]), th.tensor([2]))
}
g = dgl.heterograph(graph_data)
g.ntypes
#['disease', 'drug', 'gene']
g.etypes
#['interacts', 'interacts', 'treats']
g.canonical_etypes
#[('drug', 'interacts', 'drug'),
# ('drug', 'interacts', 'gene'),
# ('drug', 'treats', 'disease')]
g
#Graph(num_nodes={'disease': 3, 'drug': 3, 'gene': 4},
#      num_edges={('drug', 'interacts', 'drug'): 2,
#                 ('drug', 'interacts', 'gene'): 2,
#                 ('drug', 'treats', 'disease'): 1},
#      metagraph=[('drug', 'drug', 'interacts'),
#                 ('drug', 'gene', 'interacts'),
#                 ('drug', 'disease', 'treats')])
g.metagraph().edges()
#OutMultiEdgeDataView([('drug', 'drug'), ('drug', 'gene'), ('drug', 'disease')])

#### 複数の型の扱い
ノード、エッジのタイプが複数ある場合、メゾットを呼び出す際にタイプを指定できる。
> `g.num_nodes()`　ノード数、タイプの指定がない場合、前ノード数  
> `g.nodes()`　全ノードの出力。<span style="color:red;">複数ある場合のみタイプの指定が必要</span>  
> `g.nodes[''].data['']` or  `g.edges[''].data['']`　特定のプロパティの出力  
> `g.ntypes` or `g.etypes`　タイプ名の出力  
> `g.ndata[dgl.NTYPE]` or `g.edata[dgl.ETYPE]`　タイプIDの出力  
> `g.ndata[dgl.NID]` or `g.edata[dgl.EID]`　ノード、エッジIDの出力  

#### グラフの返還
異種グラフから同種グラフに変換できる
`hg = dgl.to_homopgeneous(g)` を使うと、トリプルだけコピーすることができ、特徴量などをコピーする場合、  
`hg = dgl.to_homopgeneous(g, edata=['he'])`のようにする。

### 1.6. GPUの使用(<span style= "color:red;">重要</span>)
GPU内で計算を行いたい場合、`g.to()`を用いて、GPUにコピーする必要がある  
GPUにコピーされたグラフに関する計算はすべてGPUで行われ、結果もGPUに保存される。

In [ ]:
import dgl
import torch as th
u, v = th.tensor([0, 1, 2]), th.tensor([2, 3, 4])
g = dgl.graph((u, v))
g.ndata['x'] = th.randn(5, 3)  # original feature is on CPU
g.device
#device(type='cpu')
cuda_g = g.to('cuda:0')  # accepts any device objects from backend framework
cuda_g.device
#device(type='cuda', index=0)
cuda_g.ndata['x'].device       # feature data is copied to GPU too
#device(type='cuda', index=0)

# A graph constructed from GPU tensors is also on GPU
u, v = u.to('cuda:0'), v.to('cuda:0')
g = dgl.graph((u, v))
g.device
#device(type='cuda', index=0)